# This notebook is just a data miner for the data
# Feel free to skip it if you want

In [2]:
from binance.client import Client
from binance.enums import *
from config import *
import pandas as pd
import numpy as np
import datetime
import time

### Function to get data from Binance

In [3]:
def get_binance_data(ticker: str, timeframe: str, period: int, apikey: str = APIKEY, secret: str = SECRET, kline_type:str = HistoricalKlinesType.FUTURES) -> pd.DataFrame:
    client = Client(apikey, secret)
    day = datetime.datetime.timestamp(datetime.datetime.now() - datetime.timedelta(days=period))
    res = client.get_historical_klines(symbol=ticker, interval=timeframe, start_str=str(day), klines_type=kline_type)
    df = pd.DataFrame(data=res)
    df = df.drop([11], axis=1)
    z = ['time', 'Open', 'High', 'Low', 'Close', 'Volume', 'closetime', "quoteassetvolume", "numtrades", "takerbuybaseassetvolume", "takerbuyquoteassetvolume"]
    df.columns = z
    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        df[col] = pd.to_numeric(df[col])
    df = df.set_index(pd.to_datetime(df['time'], unit='ms')).drop(columns=["time"])
    return df[['Open', 'High', 'Low', 'Close', 'Volume']]

In [3]:
# testing the fn
btc_spot = get_binance_data("BTCUSDT", timeframe="4h", period=600, kline_type=HistoricalKlinesType.SPOT)

In [4]:
btc_spot

,Open,High,Low,Close,Volume
time,,,,,
2023-06-20 12:00:00,26907.99,27134.73,26652.00,27071.93,8935.82439
2023-06-20 16:00:00,27071.94,28140.00,27001.73,27982.77,30469.11578
2023-06-20 20:00:00,27982.77,28402.74,27872.00,28307.99,13075.73322
2023-06-21 00:00:00,28308.00,29088.09,28257.99,28711.15,17152.45801
2023-06-21 04:00:00,28711.15,29005.00,28680.00,28872.28,11260.27734
...,...,...,...,...,...
2025-02-08 16:00:00,95843.74,96750.00,95749.88,96579.99,1536.63554
2025-02-08 20:00:00,96579.99,96712.00,96358.50,96444.74,1151.19052
2025-02-09 00:00:00,96444.75,97300.00,96360.14,97300.00,2064.37300


In [12]:
btc_futures = get_binance_data("BTCUSDT", timeframe="8h", period=600, kline_type=HistoricalKlinesType.FUTURES)

In [13]:
btc_futures

,Open,High,Low,Close,Volume
time,,,,,
2023-06-20 16:00:00,27054.3,28397.8,26983.3,28290.6,464376.000
2023-06-21 00:00:00,28290.7,29086.6,28236.4,28860.4,245413.524
2023-06-21 08:00:00,28860.4,30018.3,28783.0,29837.0,430834.149
2023-06-21 16:00:00,29836.9,30822.0,29745.5,29988.5,447226.754
2023-06-22 00:00:00,29988.5,30488.0,29880.0,30094.5,144267.750
...,...,...,...,...,...
2025-02-08 00:00:00,96454.5,96850.0,95714.4,96153.4,25581.620
2025-02-08 08:00:00,96153.4,96466.0,95600.0,95798.0,24727.086
2025-02-08 16:00:00,95798.0,96725.2,95685.6,96411.9,18741.726


### Function to get funding rate

In [7]:
def get_historical_funding_rates(symbols: list, days_from_today: int = 100, limit: int = 1000, apikey = APIKEY, secret = SECRET):
    # Get current time in milliseconds
    now = int(time.time()) * 1000
    startTime = now - days_from_today * 24 * 60 * 60 * 1000  # Calculate start time based on days
    endTime = now
    client = Client(apikey, secret)

    # Dictionary to store funding rates for all symbols
    all_data = {}

    for symbol in symbols:
        temp = []
        current_start_time = startTime

        while current_start_time < endTime:
            res = client.futures_funding_rate(symbol=symbol, startTime=current_start_time, endTime=endTime, limit=limit)
            temp.extend(res)

            # Update current_start_time to the last returned fundingTime for pagination
            if res:
                current_start_time = res[-1]['fundingTime'] + 1000  # Add 1 ms to avoid overlap
                if current_start_time + 8 * 60 * 60 * 1000 > endTime:  # If last returned fundingTime exceeds endTime, set it to endTime
                    current_start_time = endTime
            else:
                break
        # Process the funding rate data for this symbol
        df = pd.DataFrame.from_dict(temp)
        if not df.empty:
            # print(df)
            df['fundingTime'] = pd.to_datetime(df['fundingTime'], unit='ms', utc=True).dt.round('h')
            df['fundingRate'] = df['fundingRate'].astype(float)
            df.set_index('fundingTime', inplace=True)

            # Store the funding rates for each symbol
            all_data[symbol] = df['fundingRate']
        else:
            print(f"No funding rates found for {symbol}")

    # Combine all symbols' funding rates into a single DataFrame
    combined_df = pd.DataFrame(all_data)

    return combined_df

In [8]:
btc_funding = get_historical_funding_rates(["BTCUSDT"], 600)

In [9]:
btc_funding

,BTCUSDT
fundingTime,
2023-06-20 16:00:00+00:00,-0.000026
2023-06-21 00:00:00+00:00,0.000063
2023-06-21 08:00:00+00:00,0.000100
2023-06-21 16:00:00+00:00,0.000026
2023-06-22 00:00:00+00:00,0.000100
...,...
2025-02-08 00:00:00+00:00,-0.000041
2025-02-08 08:00:00+00:00,0.000011
2025-02-08 16:00:00+00:00,0.000003


### Now we established our fns work, here's the plan
1. get all the symbols that are tradable for the past 4 years (1460 days) 
2. get the funding rates for each symbol using the get_historical_funding_rates function
3. compute the basis for each symbol at each 4h interval (bases = futures - spot) using the get_binance_data fn
4. make a basis dataframe
5. save all the data to csv files

In [6]:
client = Client(APIKEY, SECRET)
all_symbols = client.futures_symbol_ticker()

In [7]:
all_symbols, len(all_symbols)
# sample
# ([{'symbol': 'ENSUSDT', 'price': '24.876', 'time': 1739022022277},
#   {'symbol': 'MINAUSDT', 'price': '0.3331000', 'time': 1739022017862},
#   {'symbol': 'BNBUSDT', 'price': '596.680', 'time': 1739022018567},

symbols = [s['symbol'] for s in all_symbols]
symbols, len(symbols)

(['GUSDT',
  'AMBUSDT',
  '1000XECUSDT',
  'BSVUSDT',
  'UNIUSDT',
  'AVAUSDT',
  'RIFUSDT',
  'MBOXUSDT',
  'ZETAUSDT',
  'DFUSDT',
  'BSWUSDT',
  'LOOMUSDT',
  'OGNUSDT',
  'BICOUSDT',
  'TURBOUSDT',
  'SANTOSUSDT',
  'XVSUSDT',
  'PNUTUSDT',
  'RUNEUSDT',
  'BNBUSDC',
  'KEYUSDT',
  'LAYERUSDT',
  'ALPHAUSDT',
  'DUSKUSDT',
  'ORDIUSDC',
  'ALTUSDT',
  'CRVUSDT',
  'GMXUSDT',
  'BERAUSDT',
  'BCHUSDT',
  'JUPUSDT',
  'COTIUSDT',
  'SFPUSDT',
  'DENTUSDT',
  'MANAUSDT',
  'RADUSDT',
  'AGLDUSDT',
  'HOTUSDT',
  'QUICKUSDT',
  'KMNOUSDT',
  'VTHOUSDT',
  'ETHFIUSDT',
  'WOOUSDT',
  'VOXELUSDT',
  'POPCATUSDT',
  'TRXUSDT',
  'AGIXUSDT',
  'DRIFTUSDT',
  'FLOWUSDT',
  'AEROUSDT',
  'BLURUSDT',
  'MOVEUSDT',
  'VIDTUSDT',
  'THETAUSDT',
  'LPTUSDT',
  'STORJUSDT',
  'LUMIAUSDT',
  'IMXUSDT',
  'ALGOUSDT',
  'STRAXUSDT',
  '1000WHYUSDT',
  'HBARUSDT',
  'MINAUSDT',
  'SHELLUSDT',
  'ARBUSDT',
  'DYMUSDT',
  'MAVIAUSDT',
  'KOMAUSDT',
  'UMAUSDT',
  'LISTAUSDT',
  'SPELLUSDT',
  'BTCUSDT'

In [ ]:
funding_rates = get_historical_funding_rates(symbols, days_from_today=1460, limit=1000)

No funding rates found for BTCUSDT_250627
No funding rates found for ETHUSDT_250627
No funding rates found for BTCUSDT_250328
No funding rates found for ETHUSDT_250328


In [21]:
funding_rates

,ONGUSDT,APTUSDT,SLERFUSDT,AGLDUSDT,BNTUSDT,ALPACAUSDT,KEYUSDT,BLURUSDT,CFXUSDT,SLPUSDT,...,MAGICUSDT,ACHUSDT,VVVUSDT,ACEUSDT,CRVUSDT,SPELLUSDT,BERAUSDT,ZILUSDT,SUSDT,ETHUSDT
fundingTime,,,,,,,,,,,,,,,,,,,,,
2021-02-10 16:00:00+00:00,NaN,NaN,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.001094,NaN,NaN,0.002380,NaN,0.000851
2021-02-11 00:00:00+00:00,NaN,NaN,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000100,NaN,NaN,0.000720,NaN,0.000719
2021-02-11 08:00:00+00:00,NaN,NaN,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.001217,NaN,NaN,0.000817,NaN,0.000660
2021-02-11 16:00:00+00:00,NaN,NaN,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.002947,NaN,NaN,0.002928,NaN,0.001806
2021-02-12 00:00:00+00:00,NaN,NaN,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.001348,NaN,NaN,0.002194,NaN,0.000921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-08 16:00:00+00:00,-0.000317,-0.000353,0.000050,-0.002566,0.000098,0.00005,0.0001,-0.000047,0.000100,0.00005,...,0.000061,0.000053,-0.001569,-0.000113,0.000100,-0.000736,-0.000654,0.000100,0.00005,0.000069
2025-02-08 20:00:00+00:00,-0.000257,NaN,0.000050,NaN,NaN,0.00005,NaN,NaN,NaN,0.00005,...,NaN,NaN,-0.000640,0.000050,NaN,NaN,-0.000132,NaN,0.00005,NaN
2025-02-09 00:00:00+00:00,-0.000117,-0.000011,0.000050,-0.001589,0.000100,0.00005,0.0001,0.000100,0.000100,0.00005,...,0.000051,0.000042,-0.000349,0.000050,0.000100,-0.000107,-0.000563,0.000100,0.00005,-0.000004


In [45]:
def get_8_hour_refresh_symbols(df,timestamp="2025-02-09 04:00:00+00:00"):
    if timestamp not in df.index:
        raise ValueError(f"Timestamp {timestamp} not found in the DataFrame index.")
    
    # Identify columns with NaN at the given timestamp
    eight_hour_symbols = [
        symbol for symbol in df.columns if pd.isna(df.loc[timestamp, symbol])
    ]
    
    return eight_hour_symbols

In [46]:
eight_hour_symbols = get_8_hour_refresh_symbols(funding_rates)
eight_hour_symbols

['APTUSDT',
 'AGLDUSDT',
 'BNTUSDT',
 'KEYUSDT',
 'BLURUSDT',
 'CFXUSDT',
 'SSVUSDT',
 'ADAUSDT',
 'GMTUSDT',
 'APEUSDT',
 'INJUSDT',
 'GLMRUSDT',
 '1INCHUSDT',
 'ETHUSDC',
 'DENTUSDT',
 'XRPUSDT',
 'ARKUSDT',
 'ONTUSDT',
 'ARPAUSDT',
 'BTCUSDT',
 'ATOMUSDT',
 'BANDUSDT',
 'CHRUSDT',
 'ARUSDT',
 'BCHUSDC',
 'NEARUSDC',
 'LEVERUSDT',
 'SOLUSDT',
 'CRVUSDC',
 'XVSUSDT',
 'COTIUSDT',
 'BICOUSDT',
 'SKLUSDT',
 'LRCUSDT',
 'KLAYUSDT',
 'BCHUSDT',
 'GMXUSDT',
 'XLMUSDT',
 'STXUSDT',
 'ALPHAUSDT',
 'QNTUSDT',
 'NEOUSDT',
 'PENDLEUSDT',
 'ICXUSDT',
 'DASHUSDT',
 'ARKMUSDT',
 'RVNUSDT',
 '1000XECUSDT',
 'FILUSDT',
 'MINAUSDT',
 'OGNUSDT',
 'KAVAUSDT',
 'XRPUSDC',
 'GRTUSDT',
 'BNBUSDT',
 'NEOUSDC',
 'ANKRUSDT',
 'UNIUSDT',
 'RLCUSDT',
 'ETHBTC',
 'TRUUSDT',
 'ETCUSDT',
 'XVGUSDT',
 'AVAXUSDC',
 'RSRUSDT',
 'ARBUSDT',
 'ENSUSDT',
 'ASTRUSDT',
 'BNXUSDT',
 'LTCUSDT',
 'AGIXUSDT',
 'BNBUSDC',
 'CTKUSDT',
 'ZECUSDT',
 'HBARUSDT',
 'IOTXUSDT',
 'IDEXUSDT',
 'EDUUSDT',
 'OPUSDT',
 'KSMUSDT',
 'GALAUS

In [47]:
funding_rates_8hr = get_historical_funding_rates(eight_hour_symbols, 1460, 1000)
funding_rates_8hr

,APTUSDT,AGLDUSDT,BNTUSDT,KEYUSDT,BLURUSDT,CFXUSDT,SSVUSDT,ADAUSDT,GMTUSDT,APEUSDT,...,MASKUSDT,MAVUSDT,ZENUSDT,ICPUSDT,MAGICUSDT,ACHUSDT,CRVUSDT,SPELLUSDT,ZILUSDT,ETHUSDT
fundingTime,,,,,,,,,,,,,,,,,,,,,
2021-02-10 16:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.000601,NaN,NaN,...,NaN,NaN,0.002188,NaN,NaN,NaN,0.001094,NaN,0.002380,0.000851
2021-02-11 00:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.001759,NaN,NaN,...,NaN,NaN,0.001013,NaN,NaN,NaN,0.000100,NaN,0.000720,0.000719
2021-02-11 08:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.001122,NaN,NaN,...,NaN,NaN,0.000459,NaN,NaN,NaN,0.001217,NaN,0.000817,0.000660
2021-02-11 16:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.002356,NaN,NaN,...,NaN,NaN,0.002852,NaN,NaN,NaN,0.002947,NaN,0.002928,0.001806
2021-02-12 00:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.001260,NaN,NaN,...,NaN,NaN,0.002121,NaN,NaN,NaN,0.001348,NaN,0.002194,0.000921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-08 00:00:00+00:00,-0.000367,-0.000097,-0.000042,0.0001,-0.000087,0.000100,0.000094,-0.000099,0.000100,-0.000120,...,0.000100,-0.000021,0.000073,0.000041,0.000089,-0.000235,0.000100,-0.000482,-0.000182,-0.000019
2025-02-08 08:00:00+00:00,-0.000323,-0.005060,0.000100,0.0001,0.000009,0.000100,-0.000063,0.000018,-0.000015,-0.000101,...,-0.000045,0.000100,0.000100,0.000008,0.000002,-0.000013,0.000100,-0.001212,0.000100,0.000095
2025-02-08 16:00:00+00:00,-0.000353,-0.002566,0.000098,0.0001,-0.000047,0.000100,0.000100,-0.000044,0.000100,-0.000071,...,0.000039,0.000100,0.000074,0.000100,0.000061,0.000053,0.000100,-0.000736,0.000100,0.000069


In [49]:
funding_rates_8hr.dropna()

,APTUSDT,AGLDUSDT,BNTUSDT,KEYUSDT,BLURUSDT,CFXUSDT,SSVUSDT,ADAUSDT,GMTUSDT,APEUSDT,...,MASKUSDT,MAVUSDT,ZENUSDT,ICPUSDT,MAGICUSDT,ACHUSDT,CRVUSDT,SPELLUSDT,ZILUSDT,ETHUSDT
fundingTime,,,,,,,,,,,,,,,,,,,,,
2024-07-01 16:00:00+00:00,0.000100,0.000100,0.000100,0.000015,0.000100,0.000100,0.000100,0.000100,0.000014,0.000083,...,0.000041,0.000100,-0.000787,0.000100,0.000100,0.000100,0.000054,0.000100,0.000100,0.000100
2024-07-02 00:00:00+00:00,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,-0.000018,0.000100,0.000100,-0.000082,...,-0.000222,-0.000488,-0.000544,0.000100,0.000064,0.000100,0.000100,0.000100,0.000020,0.000100
2024-07-02 08:00:00+00:00,0.000100,0.000058,0.000100,0.000100,0.000100,0.000100,-0.000031,0.000100,0.000100,0.000100,...,0.000100,0.000100,-0.001018,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100
2024-07-02 16:00:00+00:00,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,0.000093,...,0.000100,0.000100,-0.000258,0.000100,0.000100,-0.000378,0.000100,0.000100,0.000100,0.000100
2024-07-03 00:00:00+00:00,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,...,0.000100,0.000100,-0.000006,0.000100,0.000100,0.000077,0.000100,0.000100,0.000100,0.000100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-08 00:00:00+00:00,-0.000367,-0.000097,-0.000042,0.000100,-0.000087,0.000100,0.000094,-0.000099,0.000100,-0.000120,...,0.000100,-0.000021,0.000073,0.000041,0.000089,-0.000235,0.000100,-0.000482,-0.000182,-0.000019
2025-02-08 08:00:00+00:00,-0.000323,-0.005060,0.000100,0.000100,0.000009,0.000100,-0.000063,0.000018,-0.000015,-0.000101,...,-0.000045,0.000100,0.000100,0.000008,0.000002,-0.000013,0.000100,-0.001212,0.000100,0.000095
2025-02-08 16:00:00+00:00,-0.000353,-0.002566,0.000098,0.000100,-0.000047,0.000100,0.000100,-0.000044,0.000100,-0.000071,...,0.000039,0.000100,0.000074,0.000100,0.000061,0.000053,0.000100,-0.000736,0.000100,0.000069


In [ ]:
funding_rates_8hr.to_csv('data/funding_rates_8hr.csv')

,APTUSDT,AGLDUSDT,BNTUSDT,KEYUSDT,BLURUSDT,CFXUSDT,SSVUSDT,ADAUSDT,GMTUSDT,APEUSDT,...,MASKUSDT,MAVUSDT,ZENUSDT,ICPUSDT,MAGICUSDT,ACHUSDT,CRVUSDT,SPELLUSDT,ZILUSDT,ETHUSDT
fundingTime,,,,,,,,,,,,,,,,,,,,,
2021-02-10 16:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.000601,NaN,NaN,...,NaN,NaN,0.002188,NaN,NaN,NaN,0.001094,NaN,0.002380,0.000851
2021-02-11 00:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.001759,NaN,NaN,...,NaN,NaN,0.001013,NaN,NaN,NaN,0.000100,NaN,0.000720,0.000719
2021-02-11 08:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.001122,NaN,NaN,...,NaN,NaN,0.000459,NaN,NaN,NaN,0.001217,NaN,0.000817,0.000660
2021-02-11 16:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.002356,NaN,NaN,...,NaN,NaN,0.002852,NaN,NaN,NaN,0.002947,NaN,0.002928,0.001806
2021-02-12 00:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.001260,NaN,NaN,...,NaN,NaN,0.002121,NaN,NaN,NaN,0.001348,NaN,0.002194,0.000921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-08 00:00:00+00:00,-0.000367,-0.000097,-0.000042,0.0001,-0.000087,0.000100,0.000094,-0.000099,0.000100,-0.000120,...,0.000100,-0.000021,0.000073,0.000041,0.000089,-0.000235,0.000100,-0.000482,-0.000182,-0.000019
2025-02-08 08:00:00+00:00,-0.000323,-0.005060,0.000100,0.0001,0.000009,0.000100,-0.000063,0.000018,-0.000015,-0.000101,...,-0.000045,0.000100,0.000100,0.000008,0.000002,-0.000013,0.000100,-0.001212,0.000100,0.000095
2025-02-08 16:00:00+00:00,-0.000353,-0.002566,0.000098,0.0001,-0.000047,0.000100,0.000100,-0.000044,0.000100,-0.000071,...,0.000039,0.000100,0.000074,0.000100,0.000061,0.000053,0.000100,-0.000736,0.000100,0.000069


In [51]:
def get_4_hour_refresh_symbols(df,timestamp="2025-02-09 04:00:00+00:00"):
    if timestamp not in df.index:
        raise ValueError(f"Timestamp {timestamp} not found in the DataFrame index.")
    
    # Identify columns with NaN at the given timestamp
    eight_hour_symbols = [
        symbol for symbol in df.columns if not pd.isna(df.loc[timestamp, symbol])
    ]
    
    return eight_hour_symbols

In [12]:
def get_basis(tickers: list, days: int = 1460):
    basis_list = []  # Store individual basis series before concatenation
    start = 0
    length = len(tickers)
    for ticker in tickers:
        try:
            spot = get_binance_data(ticker=ticker, timeframe="8h", period=days, kline_type=HistoricalKlinesType.SPOT) 
            futures = get_binance_data(ticker=ticker, timeframe="8h", period=days, kline_type=HistoricalKlinesType.FUTURES)
            basis = futures['Close'] - spot['Close']
            basis.name = ticker  # Assign column name
            basis_list.append(basis)
        except Exception as e:
            print(f"Error for {ticker}: {e}")
        finally:
            start += 1
            print(f"Completed {start}/{length} tickers")
    
    # Combine all basis series into a single DataFrame
    basis_df = pd.concat(basis_list, axis=1)
    
    return basis_df


In [28]:
eight_hour_funding = pd.read_csv('./data/funding_rates_8hr.csv')
eight_hour_funding.set_index("fundingTime", inplace=True)
eight_hour_funding.tail()

,APTUSDT,AGLDUSDT,BNTUSDT,KEYUSDT,BLURUSDT,CFXUSDT,SSVUSDT,ADAUSDT,GMTUSDT,APEUSDT,...,MASKUSDT,MAVUSDT,ZENUSDT,ICPUSDT,MAGICUSDT,ACHUSDT,CRVUSDT,SPELLUSDT,ZILUSDT,ETHUSDT
fundingTime,,,,,,,,,,,,,,,,,,,,,
2025-02-08 00:00:00+00:00,-0.000367,-0.000097,-0.000042,0.0001,-0.000087,0.000100,0.000094,-0.000099,0.000100,-0.000120,...,0.000100,-0.000021,0.000073,0.000041,0.000089,-0.000235,0.0001,-0.000482,-0.000182,-0.000019
2025-02-08 08:00:00+00:00,-0.000323,-0.005060,0.000100,0.0001,0.000009,0.000100,-0.000063,0.000018,-0.000015,-0.000101,...,-0.000045,0.000100,0.000100,0.000008,0.000002,-0.000013,0.0001,-0.001212,0.000100,0.000095
2025-02-08 16:00:00+00:00,-0.000353,-0.002566,0.000098,0.0001,-0.000047,0.000100,0.000100,-0.000044,0.000100,-0.000071,...,0.000039,0.000100,0.000074,0.000100,0.000061,0.000053,0.0001,-0.000736,0.000100,0.000069
2025-02-09 00:00:00+00:00,-0.000011,-0.001589,0.000100,0.0001,0.000100,0.000100,0.000100,-0.000073,-0.000078,0.000055,...,0.000004,-0.000101,0.000100,0.000015,0.000051,0.000042,0.0001,-0.000107,0.000100,-0.000004
2025-02-09 08:00:00+00:00,-0.000044,-0.001101,-0.000041,0.0001,0.000100,0.000077,0.000014,0.000024,0.000100,-0.000106,...,0.000100,-0.000495,0.000100,-0.000020,0.000100,-0.000009,0.0001,-0.000137,0.000100,0.000055


In [18]:
symbols = eight_hour_funding.columns.tolist()
symbols

['fundingTime',
 'APTUSDT',
 'AGLDUSDT',
 'BNTUSDT',
 'KEYUSDT',
 'BLURUSDT',
 'CFXUSDT',
 'SSVUSDT',
 'ADAUSDT',
 'GMTUSDT',
 'APEUSDT',
 'INJUSDT',
 'GLMRUSDT',
 '1INCHUSDT',
 'ETHUSDC',
 'DENTUSDT',
 'XRPUSDT',
 'ARKUSDT',
 'ONTUSDT',
 'ARPAUSDT',
 'BTCUSDT',
 'ATOMUSDT',
 'BANDUSDT',
 'CHRUSDT',
 'ARUSDT',
 'BCHUSDC',
 'NEARUSDC',
 'LEVERUSDT',
 'SOLUSDT',
 'CRVUSDC',
 'XVSUSDT',
 'COTIUSDT',
 'BICOUSDT',
 'SKLUSDT',
 'LRCUSDT',
 'KLAYUSDT',
 'BCHUSDT',
 'GMXUSDT',
 'XLMUSDT',
 'STXUSDT',
 'ALPHAUSDT',
 'QNTUSDT',
 'NEOUSDT',
 'PENDLEUSDT',
 'ICXUSDT',
 'DASHUSDT',
 'ARKMUSDT',
 'RVNUSDT',
 '1000XECUSDT',
 'FILUSDT',
 'MINAUSDT',
 'OGNUSDT',
 'KAVAUSDT',
 'XRPUSDC',
 'GRTUSDT',
 'BNBUSDT',
 'NEOUSDC',
 'ANKRUSDT',
 'UNIUSDT',
 'RLCUSDT',
 'ETHBTC',
 'TRUUSDT',
 'ETCUSDT',
 'XVGUSDT',
 'AVAXUSDC',
 'RSRUSDT',
 'ARBUSDT',
 'ENSUSDT',
 'ASTRUSDT',
 'BNXUSDT',
 'LTCUSDT',
 'AGIXUSDT',
 'BNBUSDC',
 'CTKUSDT',
 'ZECUSDT',
 'HBARUSDT',
 'IOTXUSDT',
 'IDEXUSDT',
 'EDUUSDT',
 'OPUSDT',
 'KS

In [20]:
basis_df = get_basis(symbols)

Error for fundingTime: APIError(code=-1100): Illegal characters found in parameter 'symbol'; legal range is '^[A-Z0-9-_.]{1,20}$'.
Completed 1/199 tickers
Completed 2/199 tickers
Completed 3/199 tickers
Completed 4/199 tickers
Completed 5/199 tickers
Completed 6/199 tickers
Completed 7/199 tickers
Completed 8/199 tickers
Completed 9/199 tickers
Completed 10/199 tickers
Completed 11/199 tickers
Completed 12/199 tickers
Completed 13/199 tickers
Completed 14/199 tickers
Completed 15/199 tickers
Completed 16/199 tickers
Completed 17/199 tickers
Completed 18/199 tickers
Completed 19/199 tickers
Completed 20/199 tickers
Completed 21/199 tickers
Completed 22/199 tickers
Completed 23/199 tickers
Completed 24/199 tickers
Completed 25/199 tickers
Completed 26/199 tickers
Completed 27/199 tickers
Completed 28/199 tickers
Completed 29/199 tickers
Completed 30/199 tickers
Completed 31/199 tickers
Completed 32/199 tickers
Completed 33/199 tickers
Completed 34/199 tickers
Completed 35/199 tickers
Com

In [21]:
basis_df

,APTUSDT,AGLDUSDT,BNTUSDT,KEYUSDT,BLURUSDT,CFXUSDT,SSVUSDT,ADAUSDT,GMTUSDT,APEUSDT,...,MASKUSDT,MAVUSDT,ZENUSDT,ICPUSDT,MAGICUSDT,ACHUSDT,CRVUSDT,SPELLUSDT,ZILUSDT,ETHUSDT
time,,,,,,,,,,,,,,,,,,,,,
2021-02-13 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00197,NaN,NaN,...,NaN,NaN,0.465,NaN,NaN,NaN,0.0040,NaN,0.00077,3.06
2021-02-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00316,NaN,NaN,...,NaN,NaN,0.317,NaN,NaN,NaN,0.0070,NaN,0.00090,3.38
2021-02-14 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00053,NaN,NaN,...,NaN,NaN,0.139,NaN,NaN,NaN,0.0060,NaN,0.00042,3.79
2021-02-14 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00144,NaN,NaN,...,NaN,NaN,0.221,NaN,NaN,NaN,0.0020,NaN,0.00043,3.17
2021-02-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.00013,NaN,NaN,...,NaN,NaN,0.229,NaN,NaN,NaN,0.0030,NaN,0.00002,0.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-11 00:00:00,-0.0065,-0.0002,-0.00029,NaN,0.00004,-0.00001,-0.012,-0.00060,-0.00001,-0.0001,...,-0.0006,-0.00004,-0.011,-0.006,0.0002,-0.000017,-0.0005,-1.000000e-07,-0.00001,-1.47
2025-02-11 08:00:00,-0.0124,-0.0015,-0.00023,NaN,-0.00009,-0.00011,-0.009,-0.00050,0.00001,-0.0003,...,-0.0010,-0.00007,-0.014,-0.004,0.0001,-0.000020,-0.0002,-8.000000e-07,-0.00002,-1.20
2025-02-11 16:00:00,0.0030,-0.0016,-0.00043,NaN,-0.00015,-0.00004,-0.011,-0.00050,0.00001,0.0001,...,-0.0019,-0.00008,-0.014,-0.004,-0.0002,-0.000001,-0.0002,-6.000000e-07,-0.00002,-1.56


In [ ]:
# compare the dates and drop the dates that are not in basis df or eight_hour_funding


basis_df.index = pd.to_datetime(basis_df.index)
basis_df.index = basis_df.index.tz_localize("UTC")
eight_hour_funding.index = pd.to_datetime(eight_hour_funding.index)
common_dates = basis_df.index.intersection(eight_hour_funding.index)
# Filter both DataFrames to keep only the common dates
basis_df_clean = basis_df.loc[common_dates]
eight_hour_funding_clean = eight_hour_funding.loc[common_dates]

In [40]:
basis_df_clean

,APTUSDT,AGLDUSDT,BNTUSDT,KEYUSDT,BLURUSDT,CFXUSDT,SSVUSDT,ADAUSDT,GMTUSDT,APEUSDT,...,MASKUSDT,MAVUSDT,ZENUSDT,ICPUSDT,MAGICUSDT,ACHUSDT,CRVUSDT,SPELLUSDT,ZILUSDT,ETHUSDT
2021-02-13 16:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00197,NaN,NaN,...,NaN,NaN,0.465,NaN,NaN,NaN,0.0040,NaN,0.00077,3.06
2021-02-14 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00316,NaN,NaN,...,NaN,NaN,0.317,NaN,NaN,NaN,0.0070,NaN,0.00090,3.38
2021-02-14 08:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00053,NaN,NaN,...,NaN,NaN,0.139,NaN,NaN,NaN,0.0060,NaN,0.00042,3.79
2021-02-14 16:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00144,NaN,NaN,...,NaN,NaN,0.221,NaN,NaN,NaN,0.0020,NaN,0.00043,3.17
2021-02-15 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.00013,NaN,NaN,...,NaN,NaN,0.229,NaN,NaN,NaN,0.0030,NaN,0.00002,0.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-08 00:00:00+00:00,-0.0080,-0.0033,-0.00049,NaN,-0.00018,-0.00006,-0.008,-0.00050,-0.00009,-0.0006,...,-0.0024,-0.00021,-0.004,-0.005,-0.0001,-0.000025,0.0003,-2.400000e-06,-0.00001,-1.28
2025-02-08 08:00:00+00:00,0.0005,0.0003,0.00017,NaN,0.00000,-0.00011,-0.002,-0.00050,0.00000,-0.0012,...,-0.0018,-0.00001,0.000,-0.006,-0.0003,-0.000007,0.0003,-1.500000e-06,-0.00001,-1.18
2025-02-08 16:00:00+00:00,0.0011,-0.0068,-0.00028,NaN,-0.00012,-0.00006,-0.013,-0.00040,-0.00004,0.0001,...,-0.0018,-0.00008,-0.015,-0.005,-0.0003,-0.000014,-0.0007,-9.000000e-07,-0.00002,-1.47
2025-02-09 00:00:00+00:00,-0.0040,-0.0022,-0.00040,NaN,-0.00004,-0.00004,-0.006,-0.00030,-0.00011,-0.0007,...,-0.0012,0.00010,-0.012,-0.001,-0.0001,-0.000018,-0.0006,-6.000000e-07,0.00000,-1.29


In [39]:
eight_hour_funding_clean

,APTUSDT,AGLDUSDT,BNTUSDT,KEYUSDT,BLURUSDT,CFXUSDT,SSVUSDT,ADAUSDT,GMTUSDT,APEUSDT,...,MASKUSDT,MAVUSDT,ZENUSDT,ICPUSDT,MAGICUSDT,ACHUSDT,CRVUSDT,SPELLUSDT,ZILUSDT,ETHUSDT
2021-02-13 16:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.000100,NaN,NaN,...,NaN,NaN,0.002672,NaN,NaN,NaN,0.001913,NaN,0.000811,0.000947
2021-02-14 00:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.000674,NaN,NaN,...,NaN,NaN,0.002810,NaN,NaN,NaN,0.002581,NaN,0.003204,0.001108
2021-02-14 08:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.001900,NaN,NaN,...,NaN,NaN,0.002028,NaN,NaN,NaN,0.001649,NaN,0.001057,0.001768
2021-02-14 16:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.002005,NaN,NaN,...,NaN,NaN,0.002835,NaN,NaN,NaN,0.001332,NaN,0.001484,0.001243
2021-02-15 00:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.000877,NaN,NaN,...,NaN,NaN,0.001694,NaN,NaN,NaN,0.001583,NaN,0.000812,0.000795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-08 00:00:00+00:00,-0.000367,-0.000097,-0.000042,0.0001,-0.000087,0.000100,0.000094,-0.000099,0.000100,-0.000120,...,0.000100,-0.000021,0.000073,0.000041,0.000089,-0.000235,0.000100,-0.000482,-0.000182,-0.000019
2025-02-08 08:00:00+00:00,-0.000323,-0.005060,0.000100,0.0001,0.000009,0.000100,-0.000063,0.000018,-0.000015,-0.000101,...,-0.000045,0.000100,0.000100,0.000008,0.000002,-0.000013,0.000100,-0.001212,0.000100,0.000095
2025-02-08 16:00:00+00:00,-0.000353,-0.002566,0.000098,0.0001,-0.000047,0.000100,0.000100,-0.000044,0.000100,-0.000071,...,0.000039,0.000100,0.000074,0.000100,0.000061,0.000053,0.000100,-0.000736,0.000100,0.000069
2025-02-09 00:00:00+00:00,-0.000011,-0.001589,0.000100,0.0001,0.000100,0.000100,0.000100,-0.000073,-0.000078,0.000055,...,0.000004,-0.000101,0.000100,0.000015,0.000051,0.000042,0.000100,-0.000107,0.000100,-0.000004


In [41]:
# Find common columns in both DataFrames
common_columns = basis_df.columns.intersection(eight_hour_funding.columns)

# Keep only common columns in both DataFrames
basis_df_clean_cols = basis_df_clean[common_columns]
eight_hour_funding_clean_cols = eight_hour_funding_clean[common_columns]

In [42]:
basis_df_clean_cols

,APTUSDT,AGLDUSDT,BNTUSDT,KEYUSDT,BLURUSDT,CFXUSDT,SSVUSDT,ADAUSDT,GMTUSDT,APEUSDT,...,MASKUSDT,MAVUSDT,ZENUSDT,ICPUSDT,MAGICUSDT,ACHUSDT,CRVUSDT,SPELLUSDT,ZILUSDT,ETHUSDT
2021-02-13 16:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00197,NaN,NaN,...,NaN,NaN,0.465,NaN,NaN,NaN,0.0040,NaN,0.00077,3.06
2021-02-14 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00316,NaN,NaN,...,NaN,NaN,0.317,NaN,NaN,NaN,0.0070,NaN,0.00090,3.38
2021-02-14 08:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00053,NaN,NaN,...,NaN,NaN,0.139,NaN,NaN,NaN,0.0060,NaN,0.00042,3.79
2021-02-14 16:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00144,NaN,NaN,...,NaN,NaN,0.221,NaN,NaN,NaN,0.0020,NaN,0.00043,3.17
2021-02-15 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.00013,NaN,NaN,...,NaN,NaN,0.229,NaN,NaN,NaN,0.0030,NaN,0.00002,0.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-08 00:00:00+00:00,-0.0080,-0.0033,-0.00049,NaN,-0.00018,-0.00006,-0.008,-0.00050,-0.00009,-0.0006,...,-0.0024,-0.00021,-0.004,-0.005,-0.0001,-0.000025,0.0003,-2.400000e-06,-0.00001,-1.28
2025-02-08 08:00:00+00:00,0.0005,0.0003,0.00017,NaN,0.00000,-0.00011,-0.002,-0.00050,0.00000,-0.0012,...,-0.0018,-0.00001,0.000,-0.006,-0.0003,-0.000007,0.0003,-1.500000e-06,-0.00001,-1.18
2025-02-08 16:00:00+00:00,0.0011,-0.0068,-0.00028,NaN,-0.00012,-0.00006,-0.013,-0.00040,-0.00004,0.0001,...,-0.0018,-0.00008,-0.015,-0.005,-0.0003,-0.000014,-0.0007,-9.000000e-07,-0.00002,-1.47
2025-02-09 00:00:00+00:00,-0.0040,-0.0022,-0.00040,NaN,-0.00004,-0.00004,-0.006,-0.00030,-0.00011,-0.0007,...,-0.0012,0.00010,-0.012,-0.001,-0.0001,-0.000018,-0.0006,-6.000000e-07,0.00000,-1.29


In [43]:
eight_hour_funding_clean_cols

,APTUSDT,AGLDUSDT,BNTUSDT,KEYUSDT,BLURUSDT,CFXUSDT,SSVUSDT,ADAUSDT,GMTUSDT,APEUSDT,...,MASKUSDT,MAVUSDT,ZENUSDT,ICPUSDT,MAGICUSDT,ACHUSDT,CRVUSDT,SPELLUSDT,ZILUSDT,ETHUSDT
2021-02-13 16:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.000100,NaN,NaN,...,NaN,NaN,0.002672,NaN,NaN,NaN,0.001913,NaN,0.000811,0.000947
2021-02-14 00:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.000674,NaN,NaN,...,NaN,NaN,0.002810,NaN,NaN,NaN,0.002581,NaN,0.003204,0.001108
2021-02-14 08:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.001900,NaN,NaN,...,NaN,NaN,0.002028,NaN,NaN,NaN,0.001649,NaN,0.001057,0.001768
2021-02-14 16:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.002005,NaN,NaN,...,NaN,NaN,0.002835,NaN,NaN,NaN,0.001332,NaN,0.001484,0.001243
2021-02-15 00:00:00+00:00,NaN,NaN,0.000100,NaN,NaN,NaN,NaN,0.000877,NaN,NaN,...,NaN,NaN,0.001694,NaN,NaN,NaN,0.001583,NaN,0.000812,0.000795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-08 00:00:00+00:00,-0.000367,-0.000097,-0.000042,0.0001,-0.000087,0.000100,0.000094,-0.000099,0.000100,-0.000120,...,0.000100,-0.000021,0.000073,0.000041,0.000089,-0.000235,0.000100,-0.000482,-0.000182,-0.000019
2025-02-08 08:00:00+00:00,-0.000323,-0.005060,0.000100,0.0001,0.000009,0.000100,-0.000063,0.000018,-0.000015,-0.000101,...,-0.000045,0.000100,0.000100,0.000008,0.000002,-0.000013,0.000100,-0.001212,0.000100,0.000095
2025-02-08 16:00:00+00:00,-0.000353,-0.002566,0.000098,0.0001,-0.000047,0.000100,0.000100,-0.000044,0.000100,-0.000071,...,0.000039,0.000100,0.000074,0.000100,0.000061,0.000053,0.000100,-0.000736,0.000100,0.000069
2025-02-09 00:00:00+00:00,-0.000011,-0.001589,0.000100,0.0001,0.000100,0.000100,0.000100,-0.000073,-0.000078,0.000055,...,0.000004,-0.000101,0.000100,0.000015,0.000051,0.000042,0.000100,-0.000107,0.000100,-0.000004


In [48]:
basis_df_clean_cols.to_csv('data/basis_df_clean_cols.csv', index=True, index_label="time")
eight_hour_funding_clean_cols.to_csv('data/eight_hour_funding_clean_cols.csv', index=True,index_label="time")

In [8]:
def get_spot_prices(tickers: list, days: int = 1500):
    spot_list = []  # Store individual basis series before concatenation
    start = 0
    length = len(tickers)
    for ticker in tickers:
        try:
            spot = get_binance_data(ticker=ticker, timeframe="8h", period=days, kline_type=HistoricalKlinesType.SPOT) 
            # futures = get_binance_data(ticker=ticker, timeframe="8h", period=days, kline_type=HistoricalKlinesType.FUTURES)
            # basis = futures['Close'] - spot['Close']
            close = spot['Close']
            close.name = ticker  # Assign column name
            spot_list.append(close)
        except Exception as e:
            print(f"Error for {ticker}: {e}")
        finally:
            start += 1
            print(f"Completed {start}/{length} tickers")
    
    # Combine all basis series into a single DataFrame
    basis_df = pd.concat(spot_list, axis=1)
    
    return basis_df

In [9]:
spot = get_spot_prices(symbols)

Completed 1/426 tickers
Completed 2/426 tickers
Error for 1000XECUSDT: APIError(code=-1121): Invalid symbol.
Completed 3/426 tickers
Error for BSVUSDT: APIError(code=-1121): Invalid symbol.
Completed 4/426 tickers
Completed 5/426 tickers
Completed 6/426 tickers
Completed 7/426 tickers
Completed 8/426 tickers
Error for ZETAUSDT: APIError(code=-1121): Invalid symbol.
Completed 9/426 tickers
Completed 10/426 tickers
Completed 11/426 tickers
Completed 12/426 tickers
Completed 13/426 tickers
Error for BICOUSDT: HTTPSConnectionPool(host='api.binance.com', port=443): Read timed out.
Completed 14/426 tickers
Completed 15/426 tickers
Completed 16/426 tickers
Completed 17/426 tickers
Completed 18/426 tickers
Completed 19/426 tickers
Completed 20/426 tickers
Completed 21/426 tickers
Completed 22/426 tickers
Completed 23/426 tickers
Completed 24/426 tickers
Completed 25/426 tickers
Completed 26/426 tickers
Completed 27/426 tickers
Completed 28/426 tickers
Completed 29/426 tickers
Completed 30/426 

In [10]:
spot.index = pd.to_datetime(spot.index)
# common_dates = basis_df.index.intersection(eight_hour_funding.index)

In [11]:
spot

,GUSDT,AMBUSDT,UNIUSDT,AVAUSDT,RIFUSDT,MBOXUSDT,DFUSDT,BSWUSDT,LOOMUSDT,OGNUSDT,...,MTLUSDT,MEUSDT,TONUSDT,DARUSDT,NEIROUSDT,GASUSDT,UNFIUSDT,CYBERUSDT,FLUXUSDT,CHRUSDT
time,,,,,,,,,,,,,,,,,,,,,
2021-01-16 08:00:00,NaN,NaN,8.9069,1.3228,0.2050,NaN,NaN,NaN,NaN,0.1692,...,0.4467,NaN,NaN,NaN,NaN,NaN,6.3615,NaN,NaN,0.02603
2021-01-16 16:00:00,NaN,NaN,8.3908,1.4924,0.1955,NaN,NaN,NaN,NaN,0.1624,...,0.4560,NaN,NaN,NaN,NaN,NaN,6.2300,NaN,NaN,0.02615
2021-01-17 00:00:00,NaN,NaN,8.2718,1.2807,0.1853,NaN,NaN,NaN,NaN,0.1506,...,0.4155,NaN,NaN,NaN,NaN,NaN,5.9676,NaN,NaN,0.02438
2021-01-17 08:00:00,NaN,NaN,8.5153,1.4476,0.1912,NaN,NaN,NaN,NaN,0.1654,...,0.4362,NaN,NaN,NaN,NaN,NaN,6.8018,NaN,NaN,0.02730
2021-01-17 16:00:00,NaN,NaN,8.6890,1.5100,0.1841,NaN,NaN,NaN,NaN,0.1736,...,0.4469,NaN,NaN,NaN,NaN,NaN,6.7950,NaN,NaN,0.02733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-22 16:00:00,0.01992,0.00069,9.0390,0.7056,0.0567,0.1245,0.07629,0.0454,NaN,0.0801,...,0.9900,1.531,3.741,NaN,0.000412,3.998,NaN,1.752,0.4130,0.14110
2025-02-23 00:00:00,0.01991,0.00065,9.2080,0.6838,0.0571,0.1260,0.07585,0.0455,NaN,0.0814,...,0.9930,1.504,3.771,NaN,0.000403,4.018,NaN,1.785,0.4159,0.14200
2025-02-23 08:00:00,0.01954,0.00052,8.9480,0.6551,0.0553,0.1188,0.07492,0.0426,NaN,0.0884,...,0.9620,1.415,3.745,NaN,0.000378,3.911,NaN,1.683,0.3980,0.13470


In [12]:
basis = pd.read_csv('data/basis_df_clean_cols.csv')

In [13]:
basis.set_index('time', inplace=True)

In [14]:
basis

,APTUSDT,AGLDUSDT,BNTUSDT,KEYUSDT,BLURUSDT,CFXUSDT,SSVUSDT,ADAUSDT,GMTUSDT,APEUSDT,...,MASKUSDT,MAVUSDT,ZENUSDT,ICPUSDT,MAGICUSDT,ACHUSDT,CRVUSDT,SPELLUSDT,ZILUSDT,ETHUSDT
time,,,,,,,,,,,,,,,,,,,,,
2021-02-13 16:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00197,NaN,NaN,...,NaN,NaN,0.465,NaN,NaN,NaN,0.0040,NaN,0.00077,3.06
2021-02-14 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00316,NaN,NaN,...,NaN,NaN,0.317,NaN,NaN,NaN,0.0070,NaN,0.00090,3.38
2021-02-14 08:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00053,NaN,NaN,...,NaN,NaN,0.139,NaN,NaN,NaN,0.0060,NaN,0.00042,3.79
2021-02-14 16:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00144,NaN,NaN,...,NaN,NaN,0.221,NaN,NaN,NaN,0.0020,NaN,0.00043,3.17
2021-02-15 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.00013,NaN,NaN,...,NaN,NaN,0.229,NaN,NaN,NaN,0.0030,NaN,0.00002,0.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-08 00:00:00+00:00,-0.0080,-0.0033,-0.00049,NaN,-0.00018,-0.00006,-0.008,-0.00050,-0.00009,-0.0006,...,-0.0024,-0.00021,-0.004,-0.005,-0.0001,-0.000025,0.0003,-2.400000e-06,-0.00001,-1.28
2025-02-08 08:00:00+00:00,0.0005,0.0003,0.00017,NaN,0.00000,-0.00011,-0.002,-0.00050,0.00000,-0.0012,...,-0.0018,-0.00001,0.000,-0.006,-0.0003,-0.000007,0.0003,-1.500000e-06,-0.00001,-1.18
2025-02-08 16:00:00+00:00,0.0011,-0.0068,-0.00028,NaN,-0.00012,-0.00006,-0.013,-0.00040,-0.00004,0.0001,...,-0.0018,-0.00008,-0.015,-0.005,-0.0003,-0.000014,-0.0007,-9.000000e-07,-0.00002,-1.47


In [26]:
spot

,GUSDT,AMBUSDT,UNIUSDT,AVAUSDT,RIFUSDT,MBOXUSDT,DFUSDT,BSWUSDT,LOOMUSDT,OGNUSDT,...,MTLUSDT,MEUSDT,TONUSDT,DARUSDT,NEIROUSDT,GASUSDT,UNFIUSDT,CYBERUSDT,FLUXUSDT,CHRUSDT
time,,,,,,,,,,,,,,,,,,,,,
2021-01-16 08:00:00+00:00,NaN,NaN,8.9069,1.3228,0.2050,NaN,NaN,NaN,NaN,0.1692,...,0.4467,NaN,NaN,NaN,NaN,NaN,6.3615,NaN,NaN,0.02603
2021-01-16 16:00:00+00:00,NaN,NaN,8.3908,1.4924,0.1955,NaN,NaN,NaN,NaN,0.1624,...,0.4560,NaN,NaN,NaN,NaN,NaN,6.2300,NaN,NaN,0.02615
2021-01-17 00:00:00+00:00,NaN,NaN,8.2718,1.2807,0.1853,NaN,NaN,NaN,NaN,0.1506,...,0.4155,NaN,NaN,NaN,NaN,NaN,5.9676,NaN,NaN,0.02438
2021-01-17 08:00:00+00:00,NaN,NaN,8.5153,1.4476,0.1912,NaN,NaN,NaN,NaN,0.1654,...,0.4362,NaN,NaN,NaN,NaN,NaN,6.8018,NaN,NaN,0.02730
2021-01-17 16:00:00+00:00,NaN,NaN,8.6890,1.5100,0.1841,NaN,NaN,NaN,NaN,0.1736,...,0.4469,NaN,NaN,NaN,NaN,NaN,6.7950,NaN,NaN,0.02733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-22 16:00:00+00:00,0.01992,0.00069,9.0390,0.7056,0.0567,0.1245,0.07629,0.0454,NaN,0.0801,...,0.9900,1.531,3.741,NaN,0.000412,3.998,NaN,1.752,0.4130,0.14110
2025-02-23 00:00:00+00:00,0.01991,0.00065,9.2080,0.6838,0.0571,0.1260,0.07585,0.0455,NaN,0.0814,...,0.9930,1.504,3.771,NaN,0.000403,4.018,NaN,1.785,0.4159,0.14200
2025-02-23 08:00:00+00:00,0.01954,0.00052,8.9480,0.6551,0.0553,0.1188,0.07492,0.0426,NaN,0.0884,...,0.9620,1.415,3.745,NaN,0.000378,3.911,NaN,1.683,0.3980,0.13470


In [28]:

# basis.index = pd.to_datetime(basis.index)
basis.index = basis.index.tz_convert("UTC")
# spot.index = pd.to_datetime(spot.index)
# spot.index = spot.index.tz_localize("UTC")
common_dates = basis.index.intersection(spot.index)
# Filter both DataFrames to keep only the common dates
basis_clean = basis.loc[common_dates]
spot_clean = spot.loc[common_dates]

In [23]:
common_dates = basis.index.intersection(spot.index)

In [29]:
spot_clean

,GUSDT,AMBUSDT,UNIUSDT,AVAUSDT,RIFUSDT,MBOXUSDT,DFUSDT,BSWUSDT,LOOMUSDT,OGNUSDT,...,MTLUSDT,MEUSDT,TONUSDT,DARUSDT,NEIROUSDT,GASUSDT,UNFIUSDT,CYBERUSDT,FLUXUSDT,CHRUSDT
time,,,,,,,,,,,,,,,,,,,,,
2021-02-13 16:00:00+00:00,NaN,NaN,21.7738,2.4107,0.3270,NaN,NaN,NaN,NaN,0.4019,...,0.8740,NaN,NaN,NaN,NaN,NaN,27.8677,NaN,NaN,0.06693
2021-02-14 00:00:00+00:00,NaN,NaN,21.7438,2.4841,0.3061,NaN,NaN,NaN,NaN,0.3911,...,0.8447,NaN,NaN,NaN,NaN,NaN,24.3548,NaN,NaN,0.06363
2021-02-14 08:00:00+00:00,NaN,NaN,20.9569,2.2230,0.3006,NaN,NaN,NaN,NaN,0.3773,...,0.7972,NaN,NaN,NaN,NaN,NaN,22.5749,NaN,NaN,0.06307
2021-02-14 16:00:00+00:00,NaN,NaN,20.8245,2.2434,0.2840,NaN,NaN,NaN,NaN,0.3659,...,0.8260,NaN,NaN,NaN,NaN,NaN,23.1596,NaN,NaN,0.06330
2021-02-15 00:00:00+00:00,NaN,NaN,19.6109,2.1740,0.2835,NaN,NaN,NaN,NaN,0.3569,...,0.7654,NaN,NaN,NaN,NaN,NaN,22.3601,NaN,NaN,0.05957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-08 00:00:00+00:00,0.01810,0.00391,8.8580,0.6157,0.0543,0.0982,0.07969,0.0375,NaN,0.0698,...,0.8890,1.323,3.708,NaN,0.000323,3.764,NaN,1.624,0.3741,0.11760
2025-02-08 08:00:00+00:00,0.01807,0.00394,8.8910,0.6230,0.0541,0.1036,0.08227,0.0379,NaN,0.0708,...,0.8860,1.511,3.743,NaN,0.000325,3.758,NaN,1.646,0.3728,0.11900
2025-02-08 16:00:00+00:00,0.01900,0.00408,9.1390,0.6467,0.0561,0.1168,0.07891,0.0412,NaN,0.0746,...,0.9190,1.830,3.738,NaN,0.000327,3.858,NaN,1.727,0.3908,0.12590


In [30]:
spot_clean.to_csv('data/spot_price_clean.csv', index=True,index_label="time")